# **API-Einbindung mit Grok 3 Mini & Mistral Large Instruct**

In [ ]:
# Installiere die benötigten Bibliotheken
!pip install xai-sdk

# Lade das deutsche Sprachmodell für spaCy (falls später für ABSA oder Syn-Chain benötigt)
!python -m spacy download de_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 87.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install ipywidgets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.3 MB/s eta 0:00:00


## Import relevanter Bibliotheken

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
import pandas as pd
from ipywidgets import IntSlider, Button, FloatProgress, FileUpload, Textarea, VBox, Label, HBox, Output
from IPython.display import display
from io import BytesIO, StringIO
import asyncio
import nest_asyncio
from xai_sdk import Client
from xai_sdk.chat import user, system
from mistralai import Mistral
import requests
from google.colab import userdata
import os
import re
import json
import time
import platform
nest_asyncio.apply()


## Erstellung Client Objekts mit API Zugriff:

### API-Key in Google Colab als Umgebungsvariable setzen

In [ ]:
os.environ['XAI_API_KEY'] = userdata.get('XAI_API_KEY')

print('API-Key geladen (letzte 4 Zeichen):', os.environ['XAI_API_KEY'][-4:])

API-Key geladen (letzte 4 Zeichen): NTQk


Systemprompt hinzufügen, um zu bestimmen, wie das Sprachmodell die Anfrage des Users verarbeiten soll:

In [ ]:
# Erstellung Client Objekts mit API Zugriff:
client = Client(api_key=os.getenv("XAI_API_KEY")) # Annahme: API-Key wird als Umgebungsvariable gesetzt

chat = client.chat.create(model="grok-3-mini")  #Erstellung eines Chat-Objektes (global für Multi-Turn)
chat.append(system("You are Grok, an LLM and expert in aspect-based sentiment analysis for Google reviews about German nursing homes."))
# Globale Variablen
df_uploaded = None


# User Interface Komponenten
grok_title = widgets.Output()
with grok_title:
    display(HTML("<h2 style='text-align:center; color:#1a73e8;'>Aspect-based sentiment analysis with LLM</h2>"))

prompt_input = widgets.Textarea(
    placeholder="Enter your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(height="120px", width="100%")
)

ask_grok_button = widgets.Button(description="Ask Grok", button_style="primary", icon="paper-plane")
output_label = widgets.Textarea(
    placeholder="Answer comes here...",
    description="LLM:",
    disabled=True,
    layout=widgets.Layout(height="350px", width="100%")
)

separator = widgets.Output()
with separator:
    display(HTML("<hr>"))

csv_analysis_title = widgets.Output()
with csv_analysis_title:
    display(HTML("<h3 style='text-align:center; color:#34a853;'>Aspect-Based Analysis with FileUpload</h3>"))

uploader = widgets.FileUpload(
    accept=".csv",
    multiple=False,
    description='Upload CSV',
    icon="upload")

output = widgets.Output()

absa_button = widgets.Button(
    description="Start Full Analysis",
    button_style="success",
    icon="play",
    layout=widgets.Layout(width="48%")
    )

test_button = widgets.Button(
    description="Test",
    button_style="info",
    icon="flask",
    layout=widgets.Layout(width="48%")
    )

button_row = widgets.HBox([uploader,
                           test_button,
                           absa_button])

# Funktionen
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

def ask_grok(change=None):
    prompt = prompt_input.value.strip()
    if not prompt:
        output_label.value = "Please write a prompt!"
        return
    try:
        output_label.value = "Grok is thinking..."
        chat.append(user(prompt))
        response = chat.sample()
        text = response.content if hasattr(response, 'content') else str(response)
        output_label.value = text
    except Exception as e:
        output_label.value = f"Error: {e}"

def on_upload_change(change):
    global df_uploaded
    with output:
        output.clear_output()
        if uploader.value:
            uploaded_file = list(uploader.value.values())[0]
            name = uploaded_file["metadata"]["name"]
            content = uploaded_file["content"]
            print(f"{name} uploaded!")
            df_uploaded = pd.read_csv(StringIO(content.decode(smartEncoding())), index_col=False)
            print("Data Preview:")
            display(df_uploaded.head().style.hide(axis="index"))

def run_absa(test_mode=False):
    global df_uploaded
    if df_uploaded is None:
        with output:
            print("No file uploaded!")
        return

    prompt_template = prompt_input.value.strip()
    if not prompt_template:
        with output:
            print("Enter a prompt!")
        return

    # Select data
    df = df_uploaded.head(5) if test_mode else df_uploaded
    absa_results = []
    button = test_button if test_mode else absa_button
    button.description = "Running..." if test_mode else "ASBA starts..."

    with output:
        output.clear_output()
        print(f"Running on {len(df)} reviews...\n")

    for idx, row in df.iterrows():
        # Problematisches Zeichen noch vor der ABSA ersetzen
        safe_text = row["Erfahrungsbericht des Nutzers"].replace('"', '\\"').replace('\n', ' ').replace('\r', ' ')

        prompt = prompt_template.replace("{{text}}", safe_text)  # {{text}} enthält jetzt sicheren Text
        prompt = prompt.replace("{{id}}", str(row["Zufallszahl"]))

        try:
          temporary_chat = client.chat.create(model="grok-3-mini")
          temporary_chat.append(system("You are an expert in aspect-based sentiment analysis. Output ONLY the final JSON, one line, no extra text, no markdown."))
          temporary_chat.append(user(prompt))
          response = temporary_chat.sample()
          response_text = response.content if hasattr(response, "content") else str(response)

          # JSON Objekt extrahieren
          json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
          if not json_match:
              raise ValueError("Kein JSON gefunden")
          parsed = json.loads(json_match.group(0))

        except Exception as e:
            with output:
                print(f"Fehler bei ID {row['Zufallszahl']}: {e} → Fallback null")
            parsed = {"Personal": None, "Verpflegung": None, "Sicherheit": None, "Wohnen": None, "Hygiene": None}

        result = {
            "id": int(row["Zufallszahl"]),
            "text": row["Erfahrungsbericht des Nutzers"],  # hier bleibt der Originaltext (mit ")
            "Personal": parsed.get("Personal"),
            "Verpflegung": parsed.get("Verpflegung"),
            "Sicherheit": parsed.get("Sicherheit"),
            "Wohnen": parsed.get("Wohnen"),
            "Hygiene": parsed.get("Hygiene")
        }
        absa_results.append(json.dumps(result, ensure_ascii=False))

       # Live Feedback
        if test_mode:
            with output:
                print(json.dumps(result, ensure_ascii=False, indent=2))
            # HIER: Zeige Grok's Rohantwort im großen Textfeld
            output_label.value = response_text

        # Nur in full mode speichern
        if not test_mode:
          with open("grok_optimize_ev.jsonl", "w", encoding=smartEncoding()) as f:
            f.write("\n".join(absa_results))
          with output:
            print(f"\nFull analysis complete! Saved to grok_optimize_ev.jsonl ({len(absa_results)} lines)")

    button.description = "Test" if test_mode else "Start Full Analysis"

# Alle Callbacks
ask_grok_button.on_click(ask_grok)
uploader.observe(on_upload_change, names='value')
test_button.on_click(lambda x: run_absa(test_mode=True))
absa_button.on_click(lambda x: run_absa(test_mode=False))

# Wie die UI angezeigt wird
ui = widgets.VBox([
    grok_title,
    prompt_input,
    ask_grok_button,
    output_label,
    output,
    separator,
    csv_analysis_title,
    button_row
])

display(ui)

Schlüssel für Mistral-Large-Instruct speichern

In [ ]:
os.environ["MISTRAL_API_KEY"] = userdata.get("MISTRAL_API_KEY")
print("API-Key für Mistral geladen (letzte 4 Zeichen ):", os.environ["MISTRAL_API_KEY"][-4:])

API-Key für Mistral geladen (letzte 4 Zeichen ): 1df3


Systemprompt für Mistral-Large-Instruct

In [ ]:
# den API Schlüssel aus der Google Colab Umbegung aufrufen
api_key = os.environ["MISTRAL_API_KEY"]

# Stateless Mistral Chat Funktion
def mistral_chat(prompt, api_key):
    url = "https://chat-ai.academiccloud.de/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    data = {
        "model": "mistral-large-instruct",
        "messages": [
            {"role": "system", "content": "You are an expert in aspect-based sentiment analysis. Output ONLY the final JSON, one line, no extra text, no markdown."},
            {"role": "user", "content": prompt}
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Mistral API error: {response.text}")

# Hilfsfunktion zur Bereinigung der API-Antwort
def clean_json_response(response_text):
    # Markdown-Codeblöcke entfernen
    response_text = re.sub(r'```json\s*|\s*```', '', response_text.strip())
    # Backticks entfernen
    response_text = response_text.strip('`')
    # Leerzeichen am Anfang/Ende entfernen
    return response_text.strip()

df_uploaded = None

# User Interface Komponenten
mistral_title = widgets.Output()
with mistral_title:
    display(HTML("<h2 style='text-align:center; color:#1a73e8;'>Aspect-based sentiment analysis with LLM</h2>"))

prompt_input = widgets.Textarea(
    placeholder="Enter your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(height="120px", width="100%")
)

ask_mistral_button = widgets.Button(description="Ask Mistral", button_style="primary", icon="paper-plane")
output_label = widgets.Textarea(
    placeholder="Answer comes here...",
    description="LLM:",
    disabled=True,
    layout=widgets.Layout(height="350px", width="100%")
)

separator = widgets.Output()
with separator:
    display(HTML("<hr>"))

csv_analysis_title = widgets.Output()
with csv_analysis_title:
    display(HTML("<h3 style='text-align:center; color:#34a853;'>Aspect-Based Analysis with FileUpload</h3>"))

uploader = widgets.FileUpload(
    accept=".csv",
    multiple=False,
    description='Upload CSV',
    icon="upload")

output = widgets.Output()

absa_button = widgets.Button(
    description="Start Full Analysis",
    button_style="success",
    icon="play",
    layout=widgets.Layout(width="48%")
    )

test_button = widgets.Button(
    description="Test",
    button_style="info",
    icon="flask",
    layout=widgets.Layout(width="48%")
    )

button_row = widgets.HBox([uploader,
                           test_button,
                           absa_button])

def smartEncoding():
    plt = platform.system()
    if plt == "Windows":
        return "utf-8-sig"
    else:
        return "utf-8"

def ask_mistral(change=None):
    prompt = prompt_input.value.strip()
    if not prompt:
        output_label.value = "Please write a prompt!"
        return
    try:
        output_label.value = "Mistral is thinking..."
        chat.append(user(prompt))
        response = chat.sample()
        text = response.content if hasattr(response, 'content') else str(response)
        output_label.value = text
    except Exception as e:
        output_label.value = f"Error: {e}"

def on_upload_change(change):
    global df_uploaded
    with output:
        output.clear_output()
        if uploader.value:
            uploaded_file = list(uploader.value.values())[0]
            name = uploaded_file["metadata"]["name"]
            content = uploaded_file["content"]
            print(f"{name} uploaded!")
            df_uploaded = pd.read_csv(StringIO(content.decode(smartEncoding())), index_col=False)
            print("Data Preview:")
            display(df_uploaded.head().style.hide(axis="index"))


def run_absa(test_mode=False):
    global df_uploaded
    if df_uploaded is None:
        with output:
            print("No file uploaded!")
        return
    prompt_template = prompt_input.value.strip()
    if not prompt_template:
        with output:
            print("Enter a prompt!")
        return
    df = df_uploaded.head(5) if test_mode else df_uploaded
    absa_results = []
    button = test_button if test_mode else absa_button
    button.description = "Running..." if test_mode else "ASBA starts..."
    with output:
        output.clear_output()
        print(f"Running on {len(df)} reviews...\n")
    for idx, row in df.iterrows():
        safe_text = row["Erfahrungsbericht des Nutzers"].replace('"', '\\"').replace('\n', ' ').replace('\r', ' ')
        prompt = prompt_template.replace("{{text}}", safe_text).replace("{{id}}", str(row["Zufallszahl"]))
        try:
            response_text = mistral_chat(prompt, api_key)
            cleaned_response = clean_json_response(response_text)
            parsed = json.loads(cleaned_response)
        except Exception as e:
            with output:
                print(f"Fehler bei ID {row['Zufallszahl']}: {e} → Fallback null")
            parsed = {"Personal": None, "Verpflegung": None, "Sicherheit": None, "Wohnen": None, "Hygiene": None}
        result = {
            "id": int(row["Zufallszahl"]),
            "text": row["Erfahrungsbericht des Nutzers"],
            "Personal": parsed.get("Personal"),
            "Verpflegung": parsed.get("Verpflegung"),
            "Sicherheit": parsed.get("Sicherheit"),
            "Wohnen": parsed.get("Wohnen"),
            "Hygiene": parsed.get("Hygiene")
        }
        absa_results.append(json.dumps(result, ensure_ascii=False))
        if test_mode:
            with output:
                print(json.dumps(result, ensure_ascii=False, indent=2))
            output_label.value = response_text
    if not test_mode:
        with open("mistral_optimize_ev.jsonl", "w", encoding=smartEncoding()) as f:
            f.write("\n".join(absa_results))
        with output:
            print(f"\nFull analysis complete! Saved to mistral_optimize_ev.jsonl ({len(absa_results)} lines)")
    button.description = "Test" if test_mode else "Start Full Analysis"


# Alle Callbacks
ask_mistral_button.on_click(ask_mistral)
uploader.observe(on_upload_change, names='value')
test_button.on_click(lambda x: run_absa(test_mode=True))
absa_button.on_click(lambda x: run_absa(test_mode=False))

# Wie die UI angezeigt wird
ui = widgets.VBox([
    mistral_title,
    prompt_input,
    ask_mistral_button,
    output_label,
    output,
    separator,
    csv_analysis_title,
    button_row
])

display(ui)


In [ ]:
print("API Key starts with:", os.environ.get("MISTRAL_API_KEY")[:5])


API Key starts with: cfda3
